In [9]:
from flask import Flask, request
import os
import sys
import shutil
import urllib.parse
import subprocess
app = Flask(__name__)
#可执行文件绝对地址
compiler_path=sys.path[0]+"/build/SYSY-compiler"
rev_root=sys.path[0]+"/rev"
compiler_args=[compiler_path]
return_src={}
O0=True

#code version option
#对应ir ast pipe pipeline cfg-svg

@app.route('/receive_string', methods=['POST'])
def receive_string():
    data = request.get_data().decode('utf-8')
    if not data:
        return "No data received.", 400
    parsed_data = urllib.parse.parse_qs(data)
    #shutil.rmtree(parsed_data)
    file_path=rev_root+"/source_code.sy"    
    file_path_2=rev_root+"/source_code.c"
    os.makedirs(rev_root,exist_ok=True)
    for key, values in parsed_data.items():
        #处理输入的代码+命令
        if(key=="source_code"):
            with open(file_path, 'w') as f:
                for value in values:
                    f.write(value+'\n')
            with open(file_path_2, 'w') as f:
                for value in values:
                    f.write(value+'\n')
            compiler_args.append(file_path)
        if(key=="params"):
            if "O1" in values:
                O0=False
            compiler_args.append(values)

        if(key=="version"):
            a=0
    #ir
    result_1=subprocess.run(compiler_args,capture_output=True,text=True)
    with open(sys.path[0]+"/rev/source.ll", 'w') as f:
        f.write(result_1.stdout)
    with open(sys.path[0]+"/rev/source.ll", 'r') as f:
        return_src["source_code"] = f.read()
    
    #pipe
    if O0 == False:
        return_src["pipe"]=['PRE','DCE','mem2reg','cfgSimplify','Loop_Unroll','ConstProp','LoopRotate','StraightLineStrengthReduce']
    else :
        return_src["pipe"]=[]

    #pipline
    result_2=subprocess.run([compiler_path,file_path,'--mem2reg'],capture_output=True,text=True)
    with open(sys.path[0]+"/rev/mem2reg.ll", 'w') as f:
        f.write(result_2.stdout)
    with open(sys.path[0]+"/rev/mem2reg.ll", 'r') as f:
        return_src["opt_pipline_mem2reg"]=f.read()
    result_3=subprocess.run([compiler_path,file_path,'--dce'],capture_output=True,text=True)
    with open(sys.path[0]+"/rev/dce.ll", 'w') as f:
        f.write(result_3.stdout)
    with open(sys.path[0]+"/rev/dce.ll", 'r') as f:
        return_src["opt_pipline_dce"]=f.read()
    result_4=subprocess.run([compiler_path,file_path,'--constprop'],capture_output=True,text=True)
    with open(sys.path[0]+"/rev/constprop.ll", 'w') as f:
        f.write(result_4.stdout)
    with open(sys.path[0]+"/rev/constprop.ll", 'r') as f:
        return_src["opt_pipline_constprop"]=f.read()
    
    #AST
    ast_file_path = file_path[0:-3:1] + ".c"
    ast_args=["clang","-fsyntax-only","-Xclang", "-ast-dump" , ast_file_path]
    result_ast=subprocess.run(ast_args,capture_output=True,text=True)
    with open("/mnt/e/China-GodBolt-forEdu/rev/source.ast", 'w') as f:
        f.write(result_ast.stdout)
    with open("/mnt/e/China-GodBolt-forEdu/rev/source.ast", 'r') as f:
        return_src["ast"] = f.read()

    for folder_name in os.listdir(rev_root):
        folder_path = os.path.join(rev_root, folder_name)
        if os.path.isdir(folder_path):
            src=[]
            for file_name in os.listdir(folder_path):
                filepath = os.path.join(folder_path, file_name)
                if os.path.isfile(filepath):
                    with open(filepath, 'r') as f:
                        src.append(f.read())
            return_src[folder_name] = src
    
    return return_src
    
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Apr/2024 20:01:46] "POST /receive_string HTTP/1.1" 200 -
